In [ ]:
#Task 1
#(A) Every position is a "variable" denoting the robot's position . "Domain" is of each position variable that is every possible grid cell that it can move to and it must not be an obstacle.
# Constraints include being inside the grid , not passing through obstacles and moving diagonally


#(B)


from ortools.constraint_solver import pywrapcp, routing_enums_pb2
import math

GRID_SIZE = 5
START = (0, 0)
TARGET = (3, 3)
OBSTACLES = {(2, 2)}

MOVES = [(-1, -1), (-1, 1), (1, -1), (1, 1)]
MOVE_COST = math.sqrt(2)

def distance(from_index, to_index):
    from_x, from_y = divmod(from_index, GRID_SIZE)
    to_x, to_y = divmod(to_index, GRID_SIZE)

    if (to_x, to_y) in OBSTACLES:
        return float('inf')

    if abs(from_x - to_x) == 1 and abs(from_y - to_y) == 1:
        return MOVE_COST

    return float('inf')

manager = pywrapcp.RoutingIndexManager(GRID_SIZE**2, 1, START[0] * GRID_SIZE + START[1])
routing = pywrapcp.RoutingModel(manager)

transit_callback_index = routing.RegisterTransitCallback(distance)
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

def target_callback(index):
    return manager.IndexToNode(index) == TARGET[0] * GRID_SIZE + TARGET[1]

routing.AddVariableMinimizedByFinalizer(routing.NextVar(0))
routing.AddDisjunction([manager.NodeToIndex(TARGET[0] * GRID_SIZE + TARGET[1])], 0)

search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

solution = routing.SolveWithParameters(search_parameters)

if solution:
    print("Step-by-step Path:")
    index = routing.Start(0)
    path = []
    step = 1
    while not routing.IsEnd(index):
        node = manager.IndexToNode(index)
        x, y = divmod(node, GRID_SIZE)
        pos = (x + 1, y + 1)
        path.append(pos)
        print(f"Step {step}: Move to {pos}")
        step += 1
        index = solution.Value(routing.NextVar(index))
    pos = (TARGET[0] + 1, TARGET[1] + 1)
    path.append(pos)
    print(f"Step {step}: Move to {pos}")
else:
    print("No valid path found.")


#(C)


Step-by-step Path:
Step 1: Move to (1, 1)
Step 2: Move to (5, 5)
Step 3: Move to (5, 4)
Step 4: Move to (5, 3)
Step 5: Move to (5, 2)
Step 6: Move to (5, 1)
Step 7: Move to (4, 5)
Step 8: Move to (4, 4)
Step 9: Move to (4, 3)
Step 10: Move to (4, 2)
Step 11: Move to (4, 1)
Step 12: Move to (3, 5)
Step 13: Move to (3, 4)
Step 14: Move to (3, 3)
Step 15: Move to (3, 2)
Step 16: Move to (3, 1)
Step 17: Move to (2, 5)
Step 18: Move to (2, 4)
Step 19: Move to (2, 3)
Step 20: Move to (2, 2)
Step 21: Move to (2, 1)
Step 22: Move to (1, 5)
Step 23: Move to (1, 4)
Step 24: Move to (1, 3)
Step 25: Move to (1, 2)
Step 26: Move to (4, 4)


In [ ]:
from ortools.sat.python import cp_model
import numpy as np
from scipy.ndimage import label

class IslandErosionMonitor:
    def __init__(self, grid):
        self.grid = np.array(grid, dtype=int)
        self.rows, self.cols = self.grid.shape
        self.perimeter = 0

    def identify_largest_landmass(self):
        labeled_array, num_features = label(self.grid)
        if num_features == 0:
            return None
        largest_label = max(range(1, num_features + 1), key=lambda x: np.sum(labeled_array == x))
        return (labeled_array == largest_label).astype(int)

    def define_constraint_model(self, landmass):
        model = cp_model.CpModel()
        cell_vars = {}
        boundary_vars = []

        for i in range(self.rows):
            for j in range(self.cols):
                var = model.NewBoolVar(f'cell_{i}_{j}')
                model.Add(var == landmass[i, j])
                cell_vars[(i, j)] = var

        for i in range(self.rows):
            for j in range(self.cols):
                if landmass[i, j] == 1:
                    for di, dj in [(0, 1), (1, 0), (0, -1), (-1, 0)]:
                        ni, nj = i + di, j + dj
                        if ni < 0 or ni >= self.rows or nj < 0 or nj >= self.cols or landmass[ni, nj] == 0:
                            boundary_vars.append(model.NewBoolVar(f'boundary_{i}_{j}_{di}_{dj}'))
                            model.Add(boundary_vars[-1] == 1)

        return model, boundary_vars

    def solve_and_compute_perimeter(self):
        landmass = self.identify_largest_landmass()
        if landmass is None:
            return 0
        model, boundary_vars = self.define_constraint_model(landmass)
        solver = cp_model.CpSolver()
        status = solver.Solve(model)
        if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
            self.perimeter = len(boundary_vars)
        return self.perimeter

def main():
    island_grid = [
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    ]

    monitor = IslandErosionMonitor(island_grid)
    perimeter = monitor.solve_and_compute_perimeter()
    print(f"Perimeter of the largest landmass: {perimeter}")

if __name__ == "__main__":
    main()

Perimeter of the largest landmass: 20


In [ ]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np
import random
import time

class TSPSolver:
    def __init__(self, num_cities=10, random_seed=42):
        self.num_cities = num_cities
        self.cities = None
        self.distance_matrix = None
        random.seed(random_seed)
        np.random.seed(random_seed)

    def generate_cities(self, min_coord=0, max_coord=100):
        self.cities = [(random.uniform(min_coord, max_coord), random.uniform(min_coord, max_coord)) for _ in range(self.num_cities)]
        return self.cities

    def compute_distance_matrix(self):
        if self.cities is None:
            raise ValueError("Cities not defined")
        self.distance_matrix = np.zeros((self.num_cities, self.num_cities))
        for i in range(self.num_cities):
            for j in range(self.num_cities):
                if i != j:
                    x1, y1 = self.cities[i]
                    x2, y2 = self.cities[j]
                    self.distance_matrix[i][j] = np.hypot(x2 - x1, y2 - y1)
        return self.distance_matrix

    def solve(self):
        if self.distance_matrix is None:
            self.compute_distance_matrix()

        manager = pywrapcp.RoutingIndexManager(self.num_cities, 1, 0)
        routing = pywrapcp.RoutingModel(manager)

        def distance_callback(from_index, to_index):
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return int(self.distance_matrix[from_node][to_node] * 1000)

        transit_callback_index = routing.RegisterTransitCallback(distance_callback)
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
        search_parameters.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
        search_parameters.time_limit.seconds = 10

        start_time = time.time()
        solution = routing.SolveWithParameters(search_parameters)
        end_time = time.time()

        if solution:
            tour = []
            index = routing.Start(0)
            while not routing.IsEnd(index):
                tour.append(manager.IndexToNode(index))
                index = solution.Value(routing.NextVar(index))
            tour.append(tour[0])
            total_distance = sum(self.distance_matrix[tour[i]][tour[i+1]] for i in range(len(tour)-1))
            return {
                'tour': tour,
                'distance': total_distance,
                'computation_time': end_time - start_time,
                'success': True
            }
        return {
            'tour': None,
            'distance': None,
            'computation_time': end_time - start_time,
            'success': False
        }

def main():
    tsp = TSPSolver(num_cities=10)
    tsp.generate_cities()
    tsp.compute_distance_matrix()
    result = tsp.solve()

    if result['success']:
        print("Optimal tour:", result['tour'])
        print(f"Total distance: {result['distance']:.2f}")
        print(f"Computation time: {result['computation_time']:.4f} seconds")
    else:
        print("No solution found.")

if __name__ == "__main__":
    main()


Optimal tour: [0, 9, 3, 7, 2, 8, 5, 6, 1, 4, 0]
Total distance: 264.14
Computation time: 10.0002 seconds


In [ ]:
from ortools.sat.python import cp_model
import random

# Parameters
GRID_SIZE = 6
NUM_ROBOTS = 5
NUM_PACKAGES = 10

# Generate robots with random start positions
robots = [{
    'id': i,
    'start': (random.randint(0, GRID_SIZE-1), random.randint(0, GRID_SIZE-1)),
    'capacity': 10,
    'battery': 15
} for i in range(NUM_ROBOTS)]

# Generate packages with random start/destination and weights
packages = [{
    'id': i,
    'start': (random.randint(0, GRID_SIZE-1), random.randint(0, GRID_SIZE-1)),
    'end': (random.randint(0, GRID_SIZE-1), random.randint(0, GRID_SIZE-1)),
    'weight': random.randint(1, 5)
} for i in range(NUM_PACKAGES)]

# Create CSP model
model = cp_model.CpModel()
assign = {(i, j): model.NewBoolVar(f"assign_{i}_{j}")
          for i in range(NUM_ROBOTS) for j in range(NUM_PACKAGES)}

# Constraints
for j in range(NUM_PACKAGES):
    model.AddExactlyOne(assign[i, j] for i in range(NUM_ROBOTS))

for i in range(NUM_ROBOTS):
    model.Add(sum(assign[i, j] * packages[j]['weight']
              for j in range(NUM_PACKAGES)) <= robots[i]['capacity'])

    total_steps = sum(assign[i, j] * (
        abs(packages[j]['start'][0] - robots[i]['start'][0]) +
        abs(packages[j]['start'][1] - robots[i]['start'][1]) +
        abs(packages[j]['end'][0] - packages[j]['start'][0]) +
        abs(packages[j]['end'][1] - packages[j]['start'][1])
    ) for j in range(NUM_PACKAGES))
    model.Add(total_steps <= robots[i]['battery'])

# Objective: Minimize total delivery time
model.Minimize(sum(
    assign[i, j] * (
        abs(packages[j]['start'][0] - robots[i]['start'][0]) +
        abs(packages[j]['start'][1] - robots[i]['start'][1]) +
        abs(packages[j]['end'][0] - packages[j]['start'][0]) +
        abs(packages[j]['end'][1] - packages[j]['start'][1])
    ) for i in range(NUM_ROBOTS) for j in range(NUM_PACKAGES)
))

# Solve
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Output
if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    print("Optimal Delivery Plan:")
    for i in range(NUM_ROBOTS):
        assigned = [j for j in range(NUM_PACKAGES) if solver.Value(assign[i, j])]
        print(f"Robot {i} assigned packages: {assigned}")
else:
    print("No solution found")

No solution found


In [ ]:
from ortools.sat.python import cp_model

puzzle = [
    [5, 3, 0, 0, 7, 0, 0, 0, 0],
    [6, 0, 0, 1, 9, 5, 0, 0, 0],
    [0, 9, 8, 0, 0, 0, 0, 6, 0],
    [8, 0, 0, 0, 6, 0, 0, 0, 3],
    [4, 0, 0, 8, 0, 3, 0, 0, 1],
    [7, 0, 0, 0, 2, 0, 0, 0, 6],
    [0, 6, 0, 0, 0, 0, 2, 8, 0],
    [0, 0, 0, 4, 1, 9, 0, 0, 5],
    [0, 0, 0, 0, 8, 0, 0, 7, 9],
]

model = cp_model.CpModel()
primes = {2, 3, 5, 7}

cell = [
    [
        model.NewIntVar(1, 9, f'c_{i}_{j}') if puzzle[i][j] == 0
        else model.NewConstant(puzzle[i][j])
        for j in range(9)
    ]
    for i in range(9)
]

# Standard Sudoku constraints
for i in range(9):
    model.AddAllDifferent(cell[i])
    model.AddAllDifferent([cell[j][i] for j in range(9)])

for bi in range(3):
    for bj in range(3):
        model.AddAllDifferent(
            cell[i][j]
            for i in range(bi*3, bi*3+3)
            for j in range(bj*3, bj*3+3)
        )

# Diagonal constraints using modulo
main_diag_sum = model.NewIntVar(0, 9*9, 'main_diag_sum')
model.Add(main_diag_sum == sum(cell[i][i] for i in range(9)))
mod = model.NewIntVar(0, 2, 'mod')
model.AddModuloEquality(mod, main_diag_sum, 3)
model.Add(mod == 0)

anti_diag_sum = model.NewIntVar(0, 9*9, 'anti_diag_sum')
model.Add(anti_diag_sum == sum(cell[i][8-i] for i in range(9)))
mod2 = model.NewIntVar(0, 2, 'mod2')
model.AddModuloEquality(mod2, anti_diag_sum, 3)
model.Add(mod2 == 0)

# Prime adjacency constraints
for i in range(9):
    for j in range(9):
        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            ni, nj = i+dx, j+dy
            if 0 <= ni < 9 and 0 <= nj < 9:
                a, b = cell[i][j], cell[ni][nj]
                for p in primes:
                    # If a is prime, b cannot be prime
                    is_a_prime = model.NewBoolVar('')
                    model.Add(a == p).OnlyEnforceIf(is_a_prime)
                    model.Add(a != p).OnlyEnforceIf(is_a_prime.Not())

                    is_b_prime = model.NewBoolVar('')
                    model.Add(b == p).OnlyEnforceIf(is_b_prime)
                    model.Add(b != p).OnlyEnforceIf(is_b_prime.Not())

                    model.AddBoolOr([is_a_prime.Not(), is_b_prime.Not()])

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    print("Solution:")
    for row in cell:
        print([solver.Value(v) for v in row])
else:
    print("No solution found")

No solution found


In [ ]:
!pip install ortools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 72.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 10.1 MB/s eta 0:00:00
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
